## Smart Vision Attendance System (SVAS)
### Face Recognition Based Attendance with Punch In / Punch Out
#### This project implements a real-time face recognition attendance system using OpenCV. The system registers users, identifies them through live video, and records attendance with Punch-In and Punch-Out functionality.

### Problem Statement
#### Manual attendance systems are time-consuming and prone to errors. This system automates attendance marking using facial recognition, ensuring contactless and reliable identification.

### System Features:
#### Face Registration
#### Real-time Face Recognition
#### Attendance Logging
#### Punch-In / Punch-Out Confirmation
#### Duplicate Prevention

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime

### Step 1: Dataset Collection

### Step 2: Train Face Recognition Model (LBPH)
#### We train the LBPH model using collected images.

In [ ]:
dataset_path = "dataset"

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

faces = []
labels = []
label_map = {}
label_id = 0

for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    label_map[label_id] = person

    for img_name in os.listdir(person_path):
        img = cv2.imread(os.path.join(person_path, img_name), cv2.IMREAD_GRAYSCALE)
        detected = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in detected:
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (200, 200))
            faces.append(face)
            labels.append(label_id)

    label_id += 1

model = cv2.face.LBPHFaceRecognizer_create()
model.train(faces, np.array(labels))

model.save("face_model.xml")
np.save("label_map.npy", label_map)

print("Model trained successfully.")


### Step 3: Load Trained Model

In [ ]:
model = cv2.face.LBPHFaceRecognizer_create()
model.read("face_model.xml")
label_map = np.load("label_map.npy", allow_pickle=True).item()


### Step 4: Attendance System with Punch In / Punch Out
#### First detection → Punch In
#### Press O key → Punch Out

In [ ]:
import cv2
import pandas as pd
import os
from datetime import datetime

attendance_file = os.path.join(os.getcwd(), "Student_attendance.csv")


# Load or create attendance file
if os.path.exists(attendance_file):
    attendance_df = pd.read_csv(attendance_file)
else:
    attendance_df = pd.DataFrame(columns=["Name", "Punch In", "Punch Out"])

video = cv2.VideoCapture(0)

print("System running... ESC to exit")

while True:
    ret, frame = video.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    status = ""
    detected_name = None

    for (x, y, w, h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
        label, confidence = model.predict(face)

        if confidence < 55:
            name = label_map[label]
            detected_name = name
        else:
            name = "Unknown"

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, f"{name}", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    # Attendance logic outside face loop
    if detected_name is not None:
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print("Detected:", detected_name)

        if detected_name not in attendance_df["Name"].values:
            print("Saving Punch In for:", detected_name)
            attendance_df.loc[len(attendance_df)] = [detected_name, now, ""]
            attendance_df = attendance_df.copy()
            attendance_df.to_csv(attendance_file, index=False)
            status = "Punch In Recorded"

        else:
            idx = attendance_df[attendance_df["Name"] == detected_name].index[-1]

            if attendance_df.at[idx, "Punch Out"] == "":
                status = "Press O to Checkout"
            else:
                status = "Session Completed"

    cv2.putText(frame, status, (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)

    cv2.imshow("SVAS Attendance", frame)

    # KEY DETECTION AFTER DISPLAY
    key = cv2.waitKey(1) & 0xFF

    # Punch Out confirmation
    if detected_name is not None and key == ord('o'):
        idx = attendance_df[attendance_df["Name"] == detected_name].index[-1]
        if attendance_df.at[idx, "Punch Out"] == "":
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print("Saving Punch Out for:", detected_name)
            attendance_df.at[idx, "Punch Out"] = now
            attendance_df = attendance_df.copy()
            attendance_df.to_csv(attendance_file, index=False)

    if key == 27:
        break

video.release()
cv2.destroyAllWindows()
print("Camera released safely")


In [ ]:
import os
print("Current working folder:\n", os.getcwd())

In [ ]:
import pandas as pd

df = pd.DataFrame([["Test", "10:00", ""]], columns=["Name", "Punch In", "Punch Out"])
df.to_csv("Student_attendance.csv", index=False)

print("File created manually.")

In [7]:
import os
os.getcwd()


'C:\\Users\\Rajesh Kinjarapu'